In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm

### 归一化数组、求解两个向量之间的夹角

In [2]:
def normalize(v):
    
    # 判断为一维还是二维数组
    if np.array(v).ndim == 1:
        vectorFlag = True
    else:
        vectorFlag = False
        
    # 转为二维数组方便操作
    v = np.double(np.atleast_2d(v)).copy()
    length = norm(v, axis=1)
    v[length!=0] = (v[length!=0].T/length[length!=0]).T
    if vectorFlag:
        v = v.ravel()
    return v

def angle(v1,v2):
    
    v1 = np.array(v1)
    v2 = np.array(v2)
    
    if v1.ndim < v2.ndim:
        v1, v2 = v2, v1
    n1 = normalize(v1)
    n2 = normalize(v2)
    if v2.ndim == 1:
        angle = np.arccos(n1.dot(n2))
    else:
        angle = np.arccos(list(map(np.dot, n1, n2)))
    return angle

In [6]:
# v1和v2夹角为90 deg
v1 = [2, 0, 0]
v2 = [0, 3, 0]
print(normalize([v1,v2]))
print(angle(v1,v2)/np.pi*180)

[[1. 0. 0.]
 [0. 1. 0.]]
90.0


### 四元数操作

#### 四元数定义：

$\boldsymbol{q}=q_w+iq_x+jq_y+kq_z,\ i^2=j^2=k^2=-1,ij=k,ji=-k,jk=i,ki=j$
$\boldsymbol{q}=[q_w,\,q_x,\,q_y,\,q_z]=[q_w,\boldsymbol{q_v}]=[\cos(\theta/2),\,n_x\sin(\theta/2),\,n_y\sin(\theta/2),\,n_z\sin(\theta/2)]$

可理解为某一向量绕着$[n_x,n_y,n_z]$绕旋转$\theta$角度的旋转变换。$\boldsymbol{q}$和$-\boldsymbol{q}$表示同一旋转。

单位四元数：四元数的模为1，即

$q_w^2+q_x^2+q_y^2+q_z^2=1$

In [7]:
# 求解表示从v1旋转到v2的最短路径的四元数
def qrotate(v1,v2):
    
    n = normalize(np.cross(v1,v2))
    n = np.atleast_2d(n)
    
    # 处理NaN
    nanindex = np.isnan(n[:,0])
    n[nanindex,:] = 0
    
    # 计算两个向量之间的夹角，求解向量部分（虚部）的四元数
    angle12 = angle(v1,v2)
    q = (n.T*np.sin(angle12/2.)).T
    if q.shape[0]==1:
        q = q.flatten()
        
    return q

def unit_q(inData):
    inData = np.atleast_2d(inData)
    (m,n) = inData.shape
    if (n!=3)&(n!=4):
        raise ValueError('Quaternion must have 3 or 4 columns')
    if n == 3:
        qLength = 1-np.sum(inData**2,1)
        numLimit = 1e-12
        # 检查可能存在的由于计算机精度导致的问题
        if np.min(qLength) < -numLimit:
            raise ValueError('Quaternion is too long!')
        else:
            # 修复数值精度问题
            qLength[qLength<0] = 0
        outData = np.hstack((np.c_[np.sqrt(qLength)], inData))
        
    else:
        outData = inData
        
    return outData

In [10]:
#将v1旋转至v2：绕z轴旋转90 deg
q1 = qrotate(v1,v2)
q2 = unit_q(q1)
print(q1, q2)

[0.         0.         0.70710678] [[0.70710678 0.         0.         0.70710678]]


### 四元素运算
乘法：可等效为矩阵与向量想乘，一般不可逆。

$\boldsymbol{p}\otimes\vec{q}=[p_wq_w-\boldsymbol{p_v}^T\boldsymbol{q_v},\,p_w\boldsymbol{q_v}+q_w\boldsymbol{p_v}+\boldsymbol{p_v}\times\boldsymbol{q_v}]$


共轭：虚部取为相反数

$\boldsymbol{q}^*=q_w-iq_x-jq_y-kq_z$

逆：

$\boldsymbol{q}^{-1}=\boldsymbol{q}^*/||\boldsymbol{q}||^2$,

$(\boldsymbol{q_a}\otimes\boldsymbol{q_b})^{-1}=\boldsymbol{q_b}^{-1}\otimes\boldsymbol{q_a}^{-1}$
单位四元数的逆即为共轭。

In [13]:
def q_mult(p,q):

    flag3D = False
    p = np.atleast_2d(p)
    q = np.atleast_2d(q)
    if p.shape[1]==3 & q.shape[1]==3:
        flag3D = True

    if len(p) != len(q):
        assert (len(p)==1 or len(q)==1), \
            'Both arguments in the quaternion multiplication must have the same number of rows, unless one has only one row.'

    p = unit_q(p).T
    q = unit_q(q).T
    
    if np.prod(np.shape(p)) > np.prod(np.shape(q)):
        r=np.zeros(np.shape(p))
    else:
        r=np.zeros(np.shape(q))

    r[0] = p[0]*q[0] - p[1]*q[1] - p[2]*q[2] - p[3]*q[3]
    r[1] = p[1]*q[0] + p[0]*q[1] + p[2]*q[3] - p[3]*q[2]
    r[2] = p[2]*q[0] + p[0]*q[2] + p[3]*q[1] - p[1]*q[3]
    r[3] = p[3]*q[0] + p[0]*q[3] + p[1]*q[2] - p[2]*q[1]

    if flag3D:
        # 使得相乘后的四元数的q_w大于0
        r[:,r[0]<0] = -r[:,r[0]<0]
        r = r[1:]

    r = r.T
    return r

def q_inv(q):
    
    q = np.atleast_2d(q)
    if q.shape[1]==3:
        return -q
    else:
        qLength = np.sum(q**2, 1)
        qConj = q * np.r_[1, -1,-1,-1]
        return (qConj.T / qLength).T


In [15]:
p = [0.70710678, 0., 0., 0.70710678]
q = [0., 1., 0, 0]
print(q_mult(p,q), q_inv(p))

[[0.         0.70710678 0.70710678 0.        ]] [[ 0.70710678 -0.         -0.         -0.70710678]]


#### 四元素与欧拉角或旋转矩阵之间的转换
欧拉角有多种定义，根据定义的欧拉角不同有不同的转换公式，此处考虑惯性导航坐标系中的“航向-俯仰-横滚”角。
先绕Z轴旋转，再绕Y轴旋转，最后绕X旋转。
注意有地方会写Z-X-Y的绕转顺序，这跟X-Y轴的定义有关，即机头沿着哪条轴。

四元数到欧拉角：

$R_{yx} = 2(q_x\,q_y + q_w\,q_z), $

$R_{zx} = 2(q_x\,q_z - q_w\,q_y), $

$R_{yz} = 2(q_y\,q_z - q_w\,q_x), $

pitch = $\arcsin{R_{yx}}$

yaw = $-\arcsin(R_{zx} / \cos(\mathrm{pitch}))$

roll = $-\arcsin(R_{yz} / \cos(\mathrm{pitch}))$

四元数到旋转矩阵：

$\boldsymbol{R}=\left[\begin{array}{ccc}
q_{w}^{2}+q_{x}^{2}-q_{y}^{2}-q_{z}^{2} & 2 q_{x} q_{y}+2 q_{w} q_{z} & 2 q_{x} q_{z}-2 q_{w} q_{y} \\
2 q_{x} q_{y}-2 q_{w} q_{z} & 1-2 q_{x}^{2}-2 q_{z}^{2} & 2 q_{y} q_{z}+2 q_{w} q_{x} \\
2 q_{x} q_{z}+2 q_{w} q_{y} & 2 q_{y} q_{z}-2 q_{w} q_{x} & q_{w}^{2}-q_{x}^{2}-q_{y}^{2}+q_{z}^{2}
\end{array}\right]$

旋转矩阵 ($\boldsymbol{R}=\{m_{ij}\}) $到四元数：

$q_{w}=\frac{\sqrt{\operatorname{tr}(R)+1}}{2}, q_{x}=\frac{m_{23}-m_{32}}{4 q_{w}}, q_{y}=\frac{m_{31}-m_{13}}{4 q_{w}}, q_{z}=\frac{m_{12}-m_{21}}{4 q_{w}}$

In [25]:
def quat2seq(quats):
    
    quats = np.atleast_2d(quats)
    
    # 若四元数仅有虚数部分，使用归一化条件求解实数部分
    if quats.shape[1] == 3:
        quats = unit_q(quats)

    R_zx = 2 * (quats[:,1]*quats[:,3] - quats[:,0]*quats[:,2])
    R_yx = 2 * (quats[:,1]*quats[:,2] + quats[:,0]*quats[:,3])
    R_zy = 2 * (quats[:,2]*quats[:,3] + quats[:,0]*quats[:,1])
    
    phi  = -np.arcsin(R_zx)
    theta = np.arcsin(R_yx / np.cos(phi))
    psi   = np.arcsin(R_zy / np.cos(phi))
    
    sequence = np.column_stack((theta, phi, psi))
    
    return np.rad2deg(sequence)

def quat2rot(quat):

    q = unit_q(quat).T
        
    R = np.zeros((9, q.shape[1]))
    R[0] = q[0]**2 + q[1]**2 - q[2]**2 - q[3]**2
    R[1] = 2*(q[1]*q[2] - q[0]*q[3])
    R[2] = 2*(q[1]*q[3] + q[0]*q[2])
    R[3] = 2*(q[1]*q[2] + q[0]*q[3])
    R[4] = q[0]**2 - q[1]**2 + q[2]**2 - q[3]**2
    R[5] = 2*(q[2]*q[3] - q[0]*q[1])
    R[6] = 2*(q[1]*q[3] - q[0]*q[2])
    R[7] = 2*(q[2]*q[3] + q[0]*q[1])
    R[8] = q[0]**2 - q[1]**2 - q[2]**2 + q[3]**2
    
    if R.shape[1] == 1:
        return np.reshape(R, (3,3))
    else:
        return R.T
    
    
def rot2quat(rMat):
    
    if rMat.shape == (3,3) or rMat.shape == (9,):
        rMat=np.atleast_2d(rMat.ravel()).T
    else:
        rMat = rMat.T
    q = np.zeros((4, rMat.shape[1]))
    
    R11 = rMat[0]
    R12 = rMat[1]
    R13 = rMat[2]
    R21 = rMat[3]
    R22 = rMat[4]
    R23 = rMat[5]
    R31 = rMat[6]
    R32 = rMat[7]
    R33 = rMat[8]
    
    # Catch small numerical inaccuracies, but produce an error for larger problems
    epsilon = 1e-10
    if np.min(np.vstack((1+R11-R22-R33, 1-R11+R22-R33, 1-R11-R22+R33))) < -epsilon:
        raise ValueError('Problems with defintion of rotation matrices')
    """
    q[1] = 0.5 * np.copysign(np.sqrt(np.abs(1+R11-R22-R33)), R32-R23)
    q[2] = 0.5 * np.copysign(np.sqrt(np.abs(1-R11+R22-R33)), R13-R31)
    q[3] = 0.5 * np.copysign(np.sqrt(np.abs(1-R11-R22+R33)), R21-R12)
    q[0] = np.sqrt(1-(q[1]**2+q[2]**2+q[3]**2))
    """
    q[0] = np.sqrt(1+R11+R22+R33)/2.
    q[1] = (R32-R23)/(4*q[0])
    q[2] = (R13-R31)/(4*q[0])
    q[3] = (R21-R12)/(4*q[0])
    
    return q.T

In [33]:
p = unit_q([0.5*np.sin(np.pi/3), -0.5*np.sin(np.pi/3),0])
seq = quat2seq(p)
rot = quat2rot(p)
p_ = rot2quat(rot)
print(p)
print(seq)
print(rot)
print(p_)

[[ 0.79056942  0.4330127  -0.4330127   0.        ]]
[[-30.96375653 -43.20833915  69.94041655]]
[[ 0.625     -0.375     -0.6846532]
 [-0.375      0.625     -0.6846532]
 [ 0.6846532  0.6846532  0.25     ]]
[[ 0.79056942  0.4330127  -0.4330127   0.        ]]


#### 四元数表示旋转
向量$\boldsymbol{p}$在四元数$\boldsymbol{q}$作用下转化为$\boldsymbol{p'}$

$\boldsymbol{p'}=\boldsymbol{q}\otimes\boldsymbol{p}\otimes\boldsymbol{q}^{-1}$

In [34]:
def rotate_vector(vector, q):
    vector = np.atleast_2d(vector)
    qvector = np.hstack((np.zeros((vector.shape[0],1)), vector))
    vRotated = q_mult(q, q_mult(qvector, q_inv(q)))
    vRotated = vRotated[:,1:]

    if min(vRotated.shape)==1:
        vRotated = vRotated.ravel()

    return vRotated

In [35]:
# 将(1,0,0)绕z轴旋转180 deg
v = [1,0,0]
q = [0,0,0,1]
rotate_vector(v,q)

array([-1.,  0.,  0.])

#### 四元数积分
微分方程：

$\frac{\mathrm{d} \boldsymbol q}{\mathrm{d} t}=\frac{1}{2}\boldsymbol q\otimes [0,\quad {\boldsymbol\omega}^T]^{T}$

积分上述方程可得：

$\boldsymbol{q_{t_{i+1}}}=\boldsymbol{q_{t_{i}}}\otimes e^{\frac{1}{2}[0,\quad\boldsymbol{\omega}^T]^T\Delta T}$

泰勒展开后得到：

$\boldsymbol{q_{t_{i+1}}}=\boldsymbol{q_{t_{i}}}\otimes{\boldsymbol{q_\omega}}$
其中
$\boldsymbol{q_\omega}=\left[\cos \left(\frac{\Delta \phi (t_i)}{2}\right),\boldsymbol{n_{t_i}}\sin \left(\frac{\Delta \phi (t_i)}{2}\right)\right] = \left[\cos \left( \frac{\left| {\boldsymbol \omega ({t_i})} \right|\Delta t}{2} \right),\frac{\boldsymbol \omega (t_i)}{\left| {\boldsymbol \omega (t_i)} \right|}\sin \left( \frac{\left| {\boldsymbol \omega ({t_i})} \right|\Delta t}{2} \right)\right]$

In [37]:
def calc_quat(omega, q0, dt, CStype):
    
    omega_2d = np.atleast_2d(omega).copy()
    dt_1d = np.atleast_1d(dt).copy()
    
    # The following is (approximately) the quaternion-equivalent of the trapezoidal integration (cumtrapz)
    if omega_2d.shape[1]>1:
        omega_2d[:-1] = 0.5*(omega_2d[:-1] + omega_2d[1:])

    omega_t = np.sqrt(np.sum(omega_2d**2, 1))
    omega_nonZero = omega_t>0

    # initialize the quaternion
    q_delta = np.zeros(omega_2d.shape)
    q_pos = np.zeros((len(omega_2d)+1,4))
    q_pos[0,:] = unit_q(q0)
    
    # magnitude of position steps
    dq_total = np.sin(omega_t[omega_nonZero]*dt_1d[omega_nonZero]/2)

    q_delta[omega_nonZero,:] = omega_2d[omega_nonZero,:] * np.tile(dq_total/omega_t[omega_nonZero], (3,1)).T

    for ii in range(len(omega_2d)):
        q1 = unit_q(q_delta[ii,:])
        q2 = q_pos[ii,:]
        if CStype == 'sf':      # 相对于space-fixed坐标系       
            qm = q_mult(q1,q2)
        elif CStype == 'bf':    # 相对于body-fixed坐标系
            qm = q_mult(q2,q1)
        else:
            print('Wrong coordinate system!')
        q_pos[ii+1,:] = qm
    return q_pos


In [41]:
# x轴角速度为pi/2 rad/s
q0 = [1,0,0,0]
omega = [np.pi/2, 0, 0]
calc_quat(omega, q0, 1, 'bf')

array([[1.        , 0.        , 0.        , 0.        ],
       [0.70710678, 0.70710678, 0.        , 0.        ]])